## Блок импорта ##

In [1]:
import numpy as np
import pandas as pd

Доступные ресурсы

In [2]:
! free --mega

               total        used        free      shared  buff/cache   available
Mem:           50322         526       46028           2        3767       49274
Swap:              0           0           0


## Реализуемый класс ##

Алгоритм для предсказывания неполадок типа М1. Основан на наборе экспертных правил, определенных эвристически на основе анализа набора данных.

*Из-за недостатка времени, были реализованы только эти.*

**Правила**

* Если превышения порога только на признаках - (ток ротора, давление масла в системе, вибрация на опоре 1,2,3, 4_продольная) - проблема в электроаппаратура
* Если превышения порога только на признаках - (тех же, что и выше + вибрация на опоре 3 продольная + температура подшипника на опоре 2) - проблема в подшипник №2
* Если превышения порога только на признаках - (ток ротора, давление масла в системе, вибрация на опоре 2, 3) - то проблема в задвижка эксг
* Если превышения порога только на признаках - (давление масла в система) - то проблема в ротор эксг*

In [3]:
class rules():
    """
    Реализован класс алгоритма, работающего на основе экспертных правил.
    """
    def __init__(self, coeff: float = 1):
        """
        Params:
        coeff: float = 1 - множитель при сравнении с 
        максимальным абсолютным значением. Является гиперпараметром, 
        настраивается для каждого датасета отдельно.
        
        """
        self.coeff = coeff
    
    def fit(self, x_normal: pd.DataFrame) -> None:
        """
        
        Метод fit берет на вход датасет с нормальным режимом работы и 
        собирает с него статистику по каждому признаку.
        
        """
        self.maxes = x_normal.abs().max()
    
    def _apply_rules(self) -> tuple:
        """
        Функция для применения экспертных правил. 
        Вызывается исключительно из метода predict, 
        т.е. по сути является private сущностью.
        
        Порядок фичей: 
        'ЭКСГАУСТЕР N. ТОК РОТОРА 1', 
        'ЭКСГАУСТЕР N. ТОК РОТОРА2',
        'ЭКСГАУСТЕР N. ТОК СТАТОРА', 
        'ЭКСГАУСТЕР N. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ',
        'ЭКСГАУСТЕР N. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 1',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 2',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 3',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 4',
        'ЭКСГАУСТЕР N. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
        """
        
        
        # Если превышения порога только на признаках - 
        # (ток ротора, давление масла в системе, вибрация на опоре 1,2,3, 4_продольная) - проблема в электроаппаратура
        #electr_anom = self.anomalies[self.anomalies.iloc[:,[0,3,10,11,12,15]]].index - pd.Timedelta(value = 2, unit = 'W') 
        electr_anom = self.anomalies[self.anomalies.iloc[:,[3,10,11,12,15]].all(axis = 1)].index - pd.Timedelta(value = 2, unit = 'W') 
        electr_anom = electr_anom[np.unique(electr_anom.date, return_index = True)[1]]
        
        #Если превышения порога только на признаках 
        # - (тех же, что и выше + вибрация на опоре 3 продольная + температура подшипника на опоре 2) - проблема в подшипник №2
        #podsh_anom = self.anomalies[self.anomalies.iloc[:,[0,3,10,11,12,15, ]]].index - pd.Timedelta(value = 2, unit = 'W')
        podsh2_anom = self.anomalies[self.anomalies.iloc[:,[3,10,11,12,15,  13,5]].all(axis = 1)].index - pd.Timedelta(value = 2, unit = 'W') 
        #1.33
        podsh2_anom = podsh2_anom[np.unique(podsh2_anom.date, return_index = True)[1]]
        
        # Если превышения порога только на признаках - 
        # (ток ротора, давление масла в системе, вибрация на опоре 2, 3) - то проблема в задвижка эксг.4
        #zadv_anom = self.anomalies[self.anomalies.iloc[:,[0,3,12,13]]].index - pd.Timedelta(value = 2, unit = 'W')
        zadv_anom = self.anomalies[self.anomalies.iloc[:,[3,12,13]].all(axis = 1)].index - pd.Timedelta(value = 2, unit = 'W')
        zadv_anom = zadv_anom[np.unique(zadv_anom.date, return_index = True)[1]]

        # Если превышения порога только на признаках - (давление масла в система) - то проблема в ротор эксг
        rot_anom = self.anomalies[self.anomalies.iloc[:,3]].index - pd.Timedelta(value = 2, unit = 'W')
        rot_anom = rot_anom[np.unique(rot_anom.date, return_index = True)[1]]
        
        return (electr_anom, podsh2_anom, zadv_anom, rot_anom)
        
    def predict(self, x_test: pd.DataFrame) -> tuple:
        """
        Функция predict возвращает временные метки для каждой неисправности.
        """
        self.anomalies = (x_test.abs() > self.coeff*self.maxes)
        res = self._apply_rules()
        return res

## Инференс модели на тестовых данных всех эксгаутеров ##

In [4]:
# Чтение имеющихся данных
X_train = pd.read_parquet('../Датасеты/X_train.parquet')
Y_train = pd.read_parquet('../Датасеты/y_train.parquet')
X_test = pd.read_parquet('../Датасеты/X_test.parquet')

# список результатов для каждого эксгаустера, shape = (6,)
res = []

for i in [4, 5, 6, 7, 8, 9]: # проход по каждому эксгаустеру
    # выберем нужных эксгаустер при помощи фильтра по столбцам
    X_train_current = X_train[X_train.columns[X_train.columns.str.find(f'ЭКСГАУСТЕР {i}') == 0]] 
    X_test_current = X_test[X_test.columns[X_test.columns.str.find(f'ЭКСГАУСТЕР {i}') == 0]]
    y_train_current = Y_train[Y_train.columns[Y_train.columns.str.find(f'Y_ЭКСГАУСТЕР А/М №{i}') == 0]]
    
    # выделим нормальный режим работы
    X_train_current = X_train_current.loc[y_train_current[y_train_current == 0].dropna().index]
    
    # создадим экземпляр класса
    rule_alg = rules(coeff=2) # был 1.33
    rule_alg.fit(X_train_current) # сохраняем статистики нормлаьного режима работы
    res.append( rule_alg.predict(X_test_current) ) # производим инференс + конкатенируем результат работы к списку

**Результат**

In [7]:
res # пользуемся coeff = 2
# эксгаустеры по списку [4, 5, 6, 7, 8, 9]
# оборудование:
# electr_anom
# podsh2_anom
# zadv_anom
# rot_anom

[(DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2022-01-19 19:06:40', '2022-03-17 08:08:10',
                 '2022-03-25 01:53:30', '2022-04-16 02:42:10',
                 '2022-04-23 08:33:00', '2023-01-21 12:32:20',
                 '2023-02-20 08:30:30'],
                dtype='datetime64[ns]', name='DT', freq=None)),
 (DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None)),
 (DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime

Примеры для разных гиперапараметров, кроме используемого выше

In [5]:
res # coeff=1.33

[(DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2022-01-19 07:19:20', '2022-03-17 08:08:10',
                 '2022-03-25 01:53:30', '2022-04-16 02:00:50',
                 '2022-04-23 08:33:00', '2022-10-26 16:56:20',
                 '2022-11-20 17:33:30', '2022-12-13 07:15:30',
                 '2022-12-26 07:07:40', '2023-01-21 12:32:20',
                 '2023-02-20 08:30:30'],
                dtype='datetime64[ns]', name='DT', freq=None)),
 (DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None)),
 (DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]',

In [5]:
res # coeff=1.1

[(DatetimeIndex(['2022-04-16 02:09:20'], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2022-04-16 02:09:20'], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2022-01-19 07:19:20', '2022-01-20 05:57:10',
                 '2022-03-17 08:08:10', '2022-03-25 01:53:30',
                 '2022-03-31 12:51:00', '2022-04-08 11:55:50',
                 '2022-04-09 23:37:50', '2022-04-10 00:00:00',
                 '2022-04-11 00:00:00', '2022-04-16 02:00:30',
                 '2022-04-17 00:15:10', '2022-04-18 00:06:20',
                 '2022-04-22 11:32:50', '2022-04-23 08:33:00',
                 '2022-04-24 20:28:00', '2022-04-25 00:00:00',
                 '2022-05-05 08:34:00', '2022-05-18 07:39:00',
                 '2022-06-02 21:59:30', '2022-08-22 11:20:30',
                 '2022-08-26 21:03:20', '2022-08-27 00:00:00',
                 '2022-09-06 09:01:30', '2022-10-11 21:24:30',
 

In [5]:
res # coeff=1

[(DatetimeIndex(['2022-04-16 02:09:20', '2022-04-18 00:37:40',
                 '2022-11-08 06:46:00', '2022-11-18 23:48:50',
                 '2022-11-19 00:00:00'],
                dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex([], dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2021-12-29 09:29:40', '2022-04-16 02:09:20',
                 '2022-04-23 08:34:10', '2022-10-11 21:28:00',
                 '2022-11-18 10:10:00', '2022-11-19 02:42:20',
                 '2022-11-21 05:32:50'],
                dtype='datetime64[ns]', name='DT', freq=None),
  DatetimeIndex(['2021-12-25 12:46:30', '2021-12-29 09:29:40',
                 '2022-01-19 07:19:20', '2022-01-20 05:57:10',
                 '2022-03-17 08:08:10', '2022-03-25 01:53:30',
                 '2022-03-31 12:51:00', '2022-04-08 11:55:50',
                 '2022-04-09 23:07:10', '2022-04-10 00:00:00',
                 '2022-04-11 00:00:00', '2022-04-16 02:00:30',
                 '2022-04-17 00:

In [11]:
list(res[3][3])

[Timestamp('2021-12-18 00:00:00'),
 Timestamp('2021-12-19 00:00:00'),
 Timestamp('2021-12-20 00:00:10'),
 Timestamp('2021-12-21 00:00:20'),
 Timestamp('2021-12-22 00:20:00'),
 Timestamp('2021-12-23 00:00:10'),
 Timestamp('2021-12-24 00:00:00'),
 Timestamp('2021-12-25 00:00:00'),
 Timestamp('2021-12-26 00:00:00'),
 Timestamp('2021-12-27 00:00:00'),
 Timestamp('2021-12-28 03:41:20'),
 Timestamp('2021-12-29 00:00:10'),
 Timestamp('2021-12-30 00:00:10'),
 Timestamp('2021-12-31 00:00:30'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-01-02 00:00:00'),
 Timestamp('2022-01-03 00:00:00'),
 Timestamp('2022-01-04 00:00:00'),
 Timestamp('2022-01-05 00:00:00'),
 Timestamp('2022-01-06 00:00:10'),
 Timestamp('2022-01-07 00:00:00'),
 Timestamp('2022-01-08 00:00:00'),
 Timestamp('2022-01-09 00:00:00'),
 Timestamp('2022-01-10 00:00:00'),
 Timestamp('2022-01-11 02:51:30'),
 Timestamp('2022-01-12 00:00:00'),
 Timestamp('2022-01-13 00:00:10'),
 Timestamp('2022-01-14 00:00:00'),
 Timestamp('2022-01-